<a href="https://colab.research.google.com/github/jeguns/Manipulacion-de-datos-con-R-y-Python/blob/main/Python/10%20-%20Res%C3%BAmenes%20estad%C3%ADsticos/U10_Resumenes_Estadisticos%20-%20Codigo%20en%20clase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Lectura

In [2]:
import pandas as pd
datos1 = pd.read_csv('/content/drive/MyDrive/Cursos/Manipulación de datos con R y Python/U10 - DATOS1.csv', encoding='latin1')
datos2 = pd.read_csv('/content/drive/MyDrive/Cursos/Manipulación de datos con R y Python/U10 - DATOS2.csv', encoding='latin1')

## Repaso

In [4]:
datos1[['DOMINIO','P209']].head()

,DOMINIO,P209
0,4,1
1,4,1
2,4,
3,4,3
4,4,


In [3]:
datos1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27813 entries, 0 to 27812
Data columns (total 38 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   AÑO        27813 non-null  int64 
 1   MES        27813 non-null  int64 
 2   CONGLOME   27813 non-null  int64 
 3   VIVIENDA   27813 non-null  int64 
 4   HOGAR      27813 non-null  int64 
 5   UBIGEO     27813 non-null  int64 
 6   DOMINIO    27813 non-null  int64 
 7   ESTRATO    27813 non-null  int64 
 8   CODPERSO   27813 non-null  int64 
 9   P201P      27813 non-null  int64 
 10  P203       27813 non-null  int64 
 11  P203A      27813 non-null  object
 12  P203B      27813 non-null  object
 13  P204       27813 non-null  object
 14  P205       27813 non-null  object
 15  P206       27813 non-null  object
 16  P207       27813 non-null  object
 17  P208A      27813 non-null  object
 18  P208B      27813 non-null  object
 19  P209       27813 non-null  object
 20  P210       27813 non-null  o

In [5]:
import numpy as np

datos_A = (
    datos1
    .assign(CODIGO=lambda x: x['CONGLOME'].astype(str) + x['VIVIENDA'].astype(str) + x['HOGAR'].astype(str) + x['CODPERSO'].astype(str)) # + : concatenar
    .loc[:, ['CODIGO', 'DOMINIO', 'P207', 'P208A', 'P209']]
    .assign(
        DOMINIO=lambda x: x['DOMINIO'].map({1: "Costa Norte",
                                            2: "Costa Centro",
                                            3: "Costa Sur",
                                            4: "Sierra Norte",
                                            5: "Sierra Centro",
                                            6: "Sierra Sur",
                                            7: "Selva",
                                            8: "Lima Metropolitana"}),
        SEXO=lambda x: x['P207'].map({'1': "Hombre", '2': "Mujer"}),
        EDAD=lambda x: pd.to_numeric(x['P208A'], errors='coerce'),
        ECIV=lambda x: x['P209'].map({'1': "Conviviente",
                                      '2': "Casado(a)",
                                      '3': "Viudo(a)",
                                      '4': "Divorciado(a)",
                                      '5': "Separado(a)",
                                      '6': "Soltero(a)"}))
    .drop(columns=['P207', 'P208A', 'P209'])
)

datos_A.head() # módulo 2 = miembros del hogar (granularidad a nivel de persona)


,CODIGO,DOMINIO,SEXO,EDAD,ECIV
0,150013111,Sierra Norte,Hombre,39.0,Conviviente
1,150013112,Sierra Norte,Mujer,38.0,Conviviente
2,150013113,Sierra Norte,Mujer,7.0,NaN
3,1500124111,Sierra Norte,Mujer,49.0,Viudo(a)
4,1500124112,Sierra Norte,Hombre,10.0,NaN


In [6]:
datos_A.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27813 entries, 0 to 27812
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CODIGO   27813 non-null  object 
 1   DOMINIO  27813 non-null  object 
 2   SEXO     26896 non-null  object 
 3   EDAD     26896 non-null  float64
 4   ECIV     21949 non-null  object 
dtypes: float64(1), object(4)
memory usage: 1.1+ MB


In [7]:
datos_B = (
    datos2
    .assign(CODIGO=lambda x: x['CONGLOME'].astype(str) + x['VIVIENDA'].astype(str) + x['HOGAR'].astype(str) + x['CODPERSO'].astype(str))
    .loc[:, ['CODIGO', 'P314A', 'P314D']]
    .rename(columns={'P314A': 'INTERNET', 'P314D': 'FREC_INT'})
    .assign(
        INTERNET=lambda x: x['INTERNET'].map({'1': "Sí", '2': "No"}).astype("category"),
        FREC_INT=lambda x: x['FREC_INT'].map({
            '1': "Una vez al día",
            '2': "Una vez a la semana",
            '3': "Una vez al mes",
            '4': "Cada 2 meses o más"}).astype("category")
    )
)

datos_B.head()  # módulo 3 = educación (granularidad a nivel de persona)

,CODIGO,INTERNET,FREC_INT
0,150013111,Sí,Una vez al día
1,150013112,Sí,Una vez al día
2,150013113,No,NaN
3,1500124111,Sí,Una vez al día
4,1500124112,No,NaN


In [10]:
datos = pd.merge(datos_A, datos_B, on="CODIGO", how="inner") # inner = ambos = intersección
datos

,CODIGO,DOMINIO,SEXO,EDAD,ECIV,INTERNET,FREC_INT
0,150013111,Sierra Norte,Hombre,39.0,Conviviente,Sí,Una vez al día
1,150013112,Sierra Norte,Mujer,38.0,Conviviente,Sí,Una vez al día
2,150013113,Sierra Norte,Mujer,7.0,NaN,No,NaN
3,1500124111,Sierra Norte,Mujer,49.0,Viudo(a),Sí,Una vez al día
4,1500124112,Sierra Norte,Hombre,10.0,NaN,No,NaN
...,...,...,...,...,...,...,...
25260,20709119113,Sierra Sur,Mujer,24.0,Soltero(a),Sí,Una vez a la semana
25261,20709120111,Sierra Sur,Hombre,42.0,Casado(a),Sí,Una vez a la semana
25262,20709120112,Sierra Sur,Mujer,38.0,Casado(a),Sí,Una vez a la semana
25263,20709120113,Sierra Sur,Hombre,11.0,NaN,Sí,Una vez a la semana


In [15]:
datos['EDAD'].mean()  # datos['EDAD'] es pandas Series (una sola dimensión, lo que en R sería un vector)

36.16073619631902

In [16]:
datos[['EDAD']].mean() # datos[['EDAD']] es pandas DataFrame (dos dimensiones)

,0
EDAD,36.160736


In [17]:
datos['EDAD'].agg(mean='mean')

,EDAD
mean,36.160736


In [18]:
print(datos[datos['DOMINIO'] == "Costa Sur"]['EDAD'].mean())
print(datos[datos['DOMINIO'] == "Costa Centro"]['EDAD'].mean())
print(datos[datos['DOMINIO'] == "Costa Norte"]['EDAD'].mean())

36.381703470031546
36.01545595054096
35.880796508456086


In [21]:
datos_grouped0 = (datos
                 .groupby('DOMINIO', as_index=False)
                 .agg(PromEdad=('EDAD', 'mean')))
datos_grouped0

,DOMINIO,PromEdad
0,Costa Centro,36.015456
1,Costa Norte,35.880797
2,Costa Sur,36.381703
3,Lima Metropolitana,38.070892
4,Selva,32.694124
5,Sierra Centro,37.074074
6,Sierra Norte,37.147541
7,Sierra Sur,39.616202


In [24]:
datos_grouped1 = (datos
                 .groupby('DOMINIO', as_index=False)
                 .agg(PromEdad=('EDAD', 'mean'))
                 .sort_values(by='PromEdad', ascending=False)
                 .reset_index(drop=True))
datos_grouped1

,DOMINIO,PromEdad
0,Sierra Sur,39.616202
1,Lima Metropolitana,38.070892
2,Sierra Norte,37.147541
3,Sierra Centro,37.074074
4,Costa Sur,36.381703
5,Costa Centro,36.015456
6,Costa Norte,35.880797
7,Selva,32.694124


In [25]:
datos_grouped2 = (datos
                 .groupby('DOMINIO', as_index=False)
                 .agg(Mediana_Edad=('EDAD', 'median'))
                 .sort_values(by='Mediana_Edad', ascending=False)
                 .reset_index(drop=True))
datos_grouped2
# El 50% de las personas en la sierra sur tiene hasta 39 años
# El 50% de las personas en la selva tiene hasta 30 años

,DOMINIO,Mediana_Edad
0,Sierra Sur,39.0
1,Lima Metropolitana,36.0
2,Sierra Centro,36.0
3,Costa Sur,35.0
4,Sierra Norte,35.0
5,Costa Norte,34.0
6,Costa Centro,33.0
7,Selva,30.0


In [26]:
datos_grouped3 = (datos
                  .groupby('DOMINIO', as_index=False)
                  .agg(Q1_Edad      = ('EDAD', lambda x: x.quantile(0.25)),
                       Mediana_Edad = ('EDAD', lambda x: x.quantile(0.5)),  # R = quantile(EDAD,0.5)
                       Mediana2     = ('EDAD', 'median'),
                       P75_Edad     = ('EDAD', lambda x: x.quantile(0.75))))
datos_grouped3
# En la costa centro, el 25% de las personas tiene hasta 17 años
# En la selva, el 75% de las personas tiene hasta 49 años; el 25% tiene más de 49 años

,DOMINIO,Q1_Edad,Mediana_Edad,Mediana2,P75_Edad
0,Costa Centro,17.0,33.0,33.0,53.0
1,Costa Norte,16.0,34.0,34.0,53.0
2,Costa Sur,18.0,35.0,35.0,52.0
3,Lima Metropolitana,19.0,36.0,36.0,54.0
4,Selva,14.0,30.0,30.0,49.0
5,Sierra Centro,16.0,36.0,36.0,56.0
6,Sierra Norte,16.0,35.0,35.0,55.0
7,Sierra Sur,19.0,39.0,39.0,58.0


In [27]:
pip install ydata_profiling

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 686.1/686.1 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.8/104.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 57.0 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27081 sha256=1d34ae5665b6664c3579ef75a368a2a8f980a192b80f7544b7d09c3412aaaef7
  Stored in directory: /root/.cache/pip/wheels/dd/91/29/a79cecb328d01739e64017b6fb9a1ab9d8cb1853098ec5966d
Successfully built htmlmin


In [28]:
pip install ydata_profiling > /dev/null 2>&1

In [30]:
from ydata_profiling import ProfileReport
profile = ProfileReport(datos.iloc[0:500,:], title="Resumen de Datos")
profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [31]:
profile.to_file("reporte_datos.html")

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]